# **Laboratorijska vježba 1**: Zadaci za samostalni rad
## **Podaci iz Svjetskog izvješća o sreći**

Svjetsko izvješće o sreći godišnja je publikacija Mreže rješenja za održivi razvoj Ujedinjenih naroda (engl. *United Nations Sustainable Development Solutions Network*). Sadrži članke i ljestvice nacionalne sreće na temelju ocjena vlastitog života ispitanika, koje izvješće također povezuje s različitim životnim čimbenicima.

U ovoj laboratorijskoj vježbi istražit ćemo sreću u različitim zemljama i povezana obilježja. Skupovi podataka koje ćemo koristiti dostupni su u *Data/happiness2020.csv* i *Data/countries_info.csv*.

U nastavku je dan sažetak stupaca (značajki) u skupu podataka:

**happines2020.csv**

*   country - naziv države
*   happiness_score - ocjena sreće
*   social_support - socijalna podrška (ublažavanje učinaka nejednakosti)
*   healthy_life_expectancy - očekivanji zdravi životni vijek
*   freedom_of_choices - sloboda donošenja životnih izbora
*   generosity - velikodušnost (milosrđe, volonteri)
*   perceptrion_of_corruption - percepcija korupcije
*   world_region - regija države u svijetu

**countries_info.csv**

*   country_name - naziv države
*   area - površina u kvadratnim miljama
*   population - broj stanovnika
*   literacy - postotak pismenih stanovnika




In [ ]:
!head Data/countries_info.csv

In [ ]:
import pandas as pd

DATA_FOLDER = 'Data/'

HAPPINESS_DATASET = DATA_FOLDER+"happiness2020.csv"
COUNTRIES_DATASET = DATA_FOLDER+"countries_info.csv"

## **Zadatak 1: Učitavanje podataka**

Učitajte 2 skupa podataka u Pandas DataFrame-ove (zvane *happiness* i *countries*) te prikažite prve retke. Koristite ispravnu metodu za učitavanje podataka i provjerite jesu li podaci u očekivanom formatu.

In [ ]:
happiness = pd.read_csv("Data/happiness2020.csv", sep=",")
happiness.head(10)

In [ ]:
countries = pd.read_csv("Data/countries_info.csv", sep=",")
countries.head(10)

## **Zadatak 2: Spajanje podataka**

Kreirajte DataFrame pod nazivom *country_features* spajanjem prethodno stvorena 2 DataFrame-a. Redak novog DataFrame-a mora opisivati sve značajke koje imamo o nekoj državi.


In [ ]:
countries.rename(columns={'country_name':'country'}, inplace=True)
countries['country'] = countries['country'].apply(str.title)
country_features = pd.merge(countries, happiness, on = 'country')
country_features



## **Zadatak 3: Gdje su ljudi sretniji?**

Ispišite prvih 10 država na temelju njihove ocjene sreće (što je veća ocjena, država je sretnija).


In [ ]:
country_features_sorted = country_features.sort_values(by='happiness_score', ascending=False)
top_10_happiest_countries = country_features_sorted.head(10)
top_10_happiest_countries

Zanima nas u kojoj su regiji svijeta ljudi sretniji.

Izradite i ispišite DataFrame s:
1.   Prosječnom ocjenom sreće
2.   Brojem zemalja za svaku svjetsku regiju.

Poredajte rezultat da biste prikazali rangiranje sreće.

In [ ]:
region_happiness = country_features.groupby('world_region').agg({'happiness_score': 'mean', 'country': 'nunique'})
region_happiness.columns = ['Average happiness score', 'Number of countries']
region_happiness = region_happiness.sort_values('Average happiness score', ascending=False)
region_happiness

Najbolje rangirana regija ima samo nekoliko država! Koje su to i koji je njihov rezultat sreće?

In [ ]:
region_data = country_features[country_features["world_region"] == "North America and ANZ"]
region_data [["country", "happiness_score"]]


## **Zadatak 4: Koliko je svijet pismen?**

Ispišite nazive država s razinom pismenosti od 100%.

Za svaku zemlju ispište naziv i regiju svijeta u formatu: {ime regije}-{ime zemlje} ({ocjena sreće}).


In [ ]:
literacy_test = "100,0"
literacy_data = country_features.loc[country_features["literacy"] == literacy_test, ["world_region", "country", "happiness_score"]]
literacy_data["region_country"] = literacy_data["world_region"] + " - " + literacy_data["country"]
selected_data = literacy_data.loc[:, ["region_country", "happiness_score"]]
selected_data["formatted_data"] = selected_data.apply(lambda x: f"{x[0]} ({x[1]})", axis=1)
display(selected_data.loc[:, "formatted_data"])

Koliki je globalni prosjek razine pismenosti?

In [ ]:
country_features["global_literacy"] = country_features["literacy"].astype(str).str.replace(',', '.').astype(float)
mean_global_literacy = country_features["global_literacy"].mean()
print("Mean global literacy rate:", mean_global_literacy)

Izračunajte udio zemalja s razinom pismenosti ispod 50%. Ispišite vrijednost u postotcima, formatiranu s 2 decimale.

In [ ]:
country_features["literacy"] = country_features["literacy"].astype(str).str.replace(",", ".").astype(float)
less_than_50_literacy = len(country_features.query('literacy <= 50.0'))
percentage = less_than_50_literacy / len(country_features) * 100
formatted_percentage = "{:.2f}".format(percentage)
print("Percentage of countries with literacy rate less than or equal to 50%: ", formatted_percentage, "%")

Ispišite neobrađeni broj i postotak nepismenog svjetskog stanovništva.

In [ ]:
number_of_illiterate = sum(country_features[country_features["literacy"] < 50.0]["population"])
total_population = sum(country_features["population"])
percentage_of_illiterate = (number_of_illiterate / total_population)*100
print("Number of people with less then 50% literacy:", number_of_illiterate)
print("Percentage of illiterate people:", percentage_of_illiterate)


## **Zadatak 5: Gustoća naseljenosti**

Dodajte DataFrame-u *country_features* novi stupac zvan *population_density* koji se dobije dijeljenjem stupca *population* sa stupcem *area*.

In [ ]:
country_features['population_density'] = country_features['population'] / country_features['area']

Koji je rezultat sreće za 3 države s najnižom gustoćom naseljenosti?

In [ ]:
lowest_density = country_features.sort_values(by='population_density', ascending=True).head(3)
display(lowest_density[['country', 'happiness_score']])

## **Zadatak 6: Zdravi i sretni?**

Iscrtajte na raspršeni dijagram (engl. *scatter plot*) ocjenu sreće (x os) i očekivano trajanje zdravog života (y os).

In [ ]:
happiness.plot.scatter(x='happiness_score', y='healthy_life_expectancy')